### Import Module

In [11]:
from tqdm import tqdm_notebook as tqdm
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import threading
import random
import time
import re

### Load Data

In [12]:
def LoadData():
    df = pd.read_excel("number.xlsx")
    return df.iloc[:, 0]

### Build URLs

In [13]:
def BuildURL(main_url, paras):
    urls = []
    for item in paras:
        urls.append(main_url.format(str(item), str(item)))
    return urls

### MultiThreading

In [14]:
def MultiThreading(urls, maximum_thread, target_func):
    num_url = 0
    num_thread = 0
    threads = []
    while (num_url < len(urls)):
        if num_thread == maximum_thread:
            for thread in threads:
                thread.join()
            threads.clear()
            num_thread = 0

        t = threading.Thread(target=target_func, args=[urls[num_url]])
        t.start()
        threads.append(t)
        num_thread += 1
        num_url += 1
        qbar.update(1)

### Scrape Data

In [15]:
def ScrapeData(url):
    time.sleep(random.randint(3,10))
    home_url = "https://www.momoshop.com.tw"
    
    message = url
    pattern = re.compile("&kw=(\d)+")
    match = pattern.search(message)
    key = match.group()[4:]
    
    imgs_url = []
    
    browser = webdriver.Edge("C:\\edgedriver_win64\\msedgedriver.exe")
    
    browser.get(url)
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    # get the top image of product
    try:
        top_img = soup.find("img", class_="jqzoom")["src"]
        imgs_url.append(top_img)
    except:
        print("Failed: Extraction of {}'s top image.".format(key))
        browser.close()
        return
        
    
    # get the iframe page
    try:
        iframe_url = home_url+soup.find("iframe", id="ifrmGoods")["src"]
        browser.get(iframe_url)
        soup = BeautifulSoup(browser.page_source, 'html.parser')
    except:
        print("Failed: Go to {}'s iframe page'".format(key))
        browser.close()
        return
    
    # get the image in iframe page
    try:
        imgs = soup.find_all("img")
        for img in imgs:
            imgs_url.append(img["src"])
    except:
        print("Failed: Extraction of {}'s iframe image.".format(key))
        browser.close()
        return
    
    store_data[key] = imgs_url
    browser.close()

### Main

In [ ]:
main_url = "https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code={}&Area=search&mdiv=403&oid=1_1&cid=index&kw={}"
store_data = {}
paras = LoadData()
urls = BuildURL(main_url, paras)
qbar = tqdm(total=len(urls))
MultiThreading(urls, 5, ScrapeData)

### Data Manipulation

In [27]:
# scrape failed
for i in paras:
    if str(i) not in store_data.keys():
        print(i)

6829702
5621965
2885961
2201970
3124085
1433974
997610
997608
997607
944537


In [29]:
# build data frame
df = pd.DataFrame.from_dict(store_data, orient="index")

In [31]:
# write to excel
df.to_excel("whai.xlsx")